In [101]:
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

In [102]:
df_stops = pd.read_csv('data/stops.txt')
df_routes = pd.read_csv('data/routes.txt')
df_stoptimes = pd.read_csv('data/stop_times.txt')
df_trips = pd.read_csv('data/trips.txt')
df_shapes = pd.read_csv('data/shapes.txt')
df_dates = pd.read_csv('data/calendar_dates.txt')

In [103]:
columns_to_drop = ['stop_headsign', 'stop_code', 'route_long_name', 'route_desc', 'route_color', 'route_text_color', 'trip_short_name']

In [104]:
result = pd.merge(df_stoptimes, df_stops, how='inner',on='stop_id')
result = pd.merge(result, df_trips, how='inner',on='trip_id')
result = pd.merge(result, df_routes, how='inner',on='route_id')
result = pd.merge(result, df_dates, how='inner',on='service_id')

In [105]:
result = result.drop(columns=columns_to_drop)

In [106]:
result['arrival_time_minutes'] = result['arrival_time'].map(lambda x: int(x[3:5]))
result['arrival_time_hours'] = result['arrival_time'].map(lambda x: int(x[:2]))
result['departure_time_minutes'] = result['departure_time'].map(lambda x: int(x[3:5]))
result['departure_time_hours'] = result['departure_time'].map(lambda x: int(x[:2]))

result['arrival_time_hours'] = result['arrival_time_hours'].apply(lambda x: x-24 if x >= 24 else x )
result['departure_time_hours'] = result['departure_time_hours'].apply(lambda x: x-24 if x >= 24 else x )

result.date = result.date.astype('str')
result['month'] = result['date'].map(lambda x: x[4:6])
result['day'] = result['date'].map(lambda x: x[6:8])
result['year'] = result['date'].map(lambda x: x[0:4])
result['date2'] = result['year'] + '-' +  result['month'] + '-' + result['day']

result['departure_time'] = result['date2'] + ' ' + result['departure_time']
result['arrival_time'] = result['date2'] + ' ' + result['arrival_time']

result['departure_time'] = result['departure_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)
result['arrival_time'] = result['arrival_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)

result['arrival_time'] = result['arrival_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
result['departure_time'] = result['departure_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

<h1>Functions</h1>

In [132]:
def get_all_departures_from_bus_stop(stop_id: int, arrival_time: str):
    time = datetime.strptime(arrival_time, '%Y-%m-%d %H:%M')
    all_deparutres = result[(result["stop_id"]==stop_id) &
       (result['departure_time'] >= time) & (result['departure_time'] <= time + timedelta(hours=24))]
    return all_deparutres
#df.sort_values(by=['col1', 'col2'])

In [134]:
tmp = get_all_departures_from_bus_stop(313,'2023-01-15 00:45')
tmp.sort_values(by=['departure_time'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
219818,403202301140057_12_403-17,2023-01-15 00:57:00,2023-01-15 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Dworzec Główny,1,403_903534,1,0,N3,700,20230114,1,57,0,57,0,01,14,2023,2023-01-14
219914,403202301142358_101_403-18,2023-01-15 01:02:00,2023-01-15 01:02:00,313,55,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903388,1,0,N3,700,20230114,1,2,1,2,1,01,14,2023,2023-01-14
219968,403202301150157_12_403-18,2023-01-15 01:57:00,2023-01-15 01:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Dworzec Główny,1,403_903534,1,0,N3,700,20230114,1,57,1,57,1,01,14,2023,2023-01-14
219534,403202301140058_101_403-15,2023-01-15 02:02:00,2023-01-15 02:02:00,313,55,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903388,1,0,N3,700,20230114,1,2,2,2,2,01,14,2023,2023-01-14
219588,403202301150248_62_403-15,2023-01-15 02:48:00,2023-01-15 02:48:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Dworzec Główny,1,403_903387,1,0,N3,700,20230114,1,48,2,48,2,01,14,2023,2023-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279509,110202301152246_22_110-41,2023-01-15 23:05:00,2023-01-15 23:05:00,313,13,1,0,Wrzeszcz PKP 06,54.381766,18.603963,110,120230115,Wrzeszcz PKP,1,110_903587,1,0,110,700,20230115,1,5,23,5,23,01,15,2023,2023-01-15
243325,162202301152247_22_162-54,2023-01-15 23:16:00,2023-01-15 23:16:00,313,19,1,0,Wrzeszcz PKP 06,54.381766,18.603963,162,120230115,Wrzeszcz PKP,1,162_903467,1,0,162,700,20230115,1,16,23,16,23,01,15,2023,2023-01-15
280200,157202301152312_32_157-51,2023-01-15 23:33:00,2023-01-15 23:33:00,313,17,1,0,Wrzeszcz PKP 06,54.381766,18.603963,157,120230115,Wrzeszcz PKP,1,157_903688,1,0,157,700,20230115,1,33,23,33,23,01,15,2023,2023-01-15
274348,403202301152348_62_403-16,2023-01-15 23:48:00,2023-01-15 23:48:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230115,Dworzec Główny,1,403_903387,1,0,N3,700,20230115,1,48,23,48,23,01,15,2023,2023-01-15


<h1>Tests</h1>